# Imports

In [1]:
import pandas as pd
import numpy as np
import os

# Structure

In [2]:
def define_paths(data_folder=None, date=None):
    if data_folder:
        shapes_df_path = os.path.join(data_folder, 'shapes.csv')
        routes_df_path = os.path.join(data_folder, 'routes.csv')
        route_versions_df_path = os.path.join(data_folder, 'route_versions.csv')
        shape_variants_df_path = os.path.join(data_folder, 'shape_variants.csv')
        shape_variant_activations_df_path = os.path.join(data_folder, 'shape_variant_activations.csv')

        temporary_changes_df_path = os.path.join(data_folder, 'temporary_changes.csv')
        return shapes_df_path, routes_df_path, route_versions_df_path, shape_variants_df_path, shape_variant_activations_df_path, temporary_changes_df_path

    if date:
        date_folder = '../data/raw/'
        routes_path = os.path.join(date_folder, date, 'routes.txt')
        trips_path = os.path.join(date_folder, date, 'trips.txt')
        shapes_path = os.path.join(date_folder, date, 'shapes.txt')
        calendar_path = os.path.join(date_folder, date, 'calendar.txt')
        calendar_dates_path = os.path.join(date_folder, date, 'calendar_dates.txt')

        return routes_path, trips_path, shapes_path, calendar_path, calendar_dates_path

    raise ValueError("Either data_folder or date must be provided.")

In [3]:
def load_txt_data(date, print_shapes=False):
    routes_path, trips_path, shapes_path, calendar_path, calendar_dates_path = define_paths(date=date)
    routes_txt = pd.read_csv(routes_path)
    trips_txt = pd.read_csv(trips_path)
    shapes_txt = pd.read_csv(shapes_path)
    calendar_dates_txt = pd.read_csv(calendar_dates_path)

    if print_shapes:
        print("Routes:", routes_txt.shape)
        print("Trips:", trips_txt.shape)
        print("Shapes:", shapes_txt.shape)
        print("Calendar Dates:", calendar_dates_txt.shape)

    try:  # Check if the file exists
        calendar_txt = pd.read_csv(calendar_path, parse_dates=['start_date', 'end_date'])
    except FileNotFoundError:
        # Make empty dataframes for the first time
        print("Calendar file not found. Creating empty dataframe.")
        calendar_txt = pd.DataFrame(columns=['service_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'start_date', 'end_date'])
        calendar_txt['start_date'] = pd.to_datetime(calendar_txt['start_date'])
        calendar_txt['end_date'] = pd.to_datetime(calendar_txt['end_date'])
    
    return routes_txt, trips_txt, shapes_txt, calendar_txt, calendar_dates_txt

In [4]:
date = '20131018'
routes_txt, trips_txt, shapes_txt, calendar_txt, calendar_dates_txt = load_txt_data(date)

In [5]:
def load_df_data(data_folder):
    shapes_df_path, routes_df_path, route_versions_df_path, shape_variants_df_path, shape_variant_activations_df_path, temporary_changes_df_path = define_paths(data_folder=data_folder)

    try:
        shapes_df = pd.read_csv(shapes_df_path)
        routes_df = pd.read_csv(routes_df_path)
        route_versions_df = pd.read_csv(route_versions_df_path, parse_dates=['valid_from', 'valid_to'])
        shape_variants_df = pd.read_csv(shape_variants_df_path)
        shape_variant_activations_df = pd.read_csv(shape_variant_activations_df_path)
        temporary_changes_df = pd.read_csv(temporary_changes_df_path)
    except FileNotFoundError:
        # Make empty dataframes for the first time
        ### shapes_df ###
        shapes_df = pd.DataFrame(columns=[
            "shape_id", "shape_pt_lat", "shape_pt_lon", "shape_pt_sequence", "shape_dist_traveled", "shape_bkk_ref"
        ])

        ### routes_df ###
        routes_df = pd.DataFrame(columns=[
            "route_id", "agency_id", "route_short_name", "route_type", "route_color", "route_text_color"
        ])

        ### route_versions_df ###
        route_versions_df = pd.DataFrame(columns=[
            "version_id", "route_id", "direction_id", "route_long_name", "route_desc",
            "valid_from", "valid_to", "main_shape_id", "trip_headsign",
            "parent_version_id", "note"
        ])
        # valid_from and valid_to be converted to datetime
        route_versions_df['valid_from'] = pd.to_datetime(route_versions_df['valid_from'])
        route_versions_df['valid_to'] = pd.to_datetime(route_versions_df['valid_to'])
        
        ### shape_variants_df ###
        shape_variants_df = pd.DataFrame(columns=[
            "shape_variant_id", "version_id", "shape_id", "trip_headsign", "is_main", "note"
        ])

        ### shape_variant_activations_df ###
        shape_variant_activations_df = pd.DataFrame(columns=[
            "date", "shape_variant_id", "exception_type"
        ])

        ### temporary_changes_df ###
        temporary_changes_df = pd.DataFrame(columns=[
            "detour_id", "route_id", "start_date", "end_date", "affects_version_id", "description"
        ])
        # Save
        shapes_df.to_csv(shapes_df_path, index=False)
        routes_df.to_csv(routes_df_path, index=False)
        route_versions_df.to_csv(route_versions_df_path, index=False)
        shape_variants_df.to_csv(shape_variants_df_path, index=False)
        shape_variant_activations_df.to_csv(shape_variant_activations_df_path, index=False)
        temporary_changes_df.to_csv(temporary_changes_df_path, index=False)

    return shapes_df, routes_df, route_versions_df, shape_variants_df, shape_variant_activations_df, temporary_changes_df

In [6]:
data_folder = '../data/processed/'
shapes_df, routes_df, route_versions_df, shape_variants_df, shape_variant_activations_df, temporary_changes_df = load_df_data(data_folder)

In [7]:
trips_df = trips_txt.copy()
calendar_df = calendar_txt.copy()
calendar_dates_df = calendar_dates_txt.copy()

## Info from calendar_df

In [8]:
calendar_df

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,A65119ASZGPP-021,0,0,0,0,0,1,0,2013-10-12,2013-10-19
1,A65120AVVGPP-021,0,0,0,0,0,0,1,2013-10-13,2013-12-22
2,A69360K1HCGPP-011,1,1,1,1,0,0,0,2013-10-28,2013-12-23
3,A69361K1PPGPP-011,0,0,0,0,1,0,0,2013-10-31,2013-10-31
4,A77793ASZSRM-021,0,0,0,0,0,1,0,2013-10-12,2013-12-14
...,...,...,...,...,...,...,...,...,...,...
1126,B01876ASZBYR-011,0,0,0,0,0,1,0,2013-10-12,2013-12-14
1127,B01879AVVBYR-011,0,0,0,0,0,0,1,2013-10-13,2013-12-22
1128,B01892ASZKP-0011,0,0,0,0,0,1,0,2013-10-12,2013-10-12
1129,B01903AVCSRM-021,1,1,1,1,0,0,1,2013-10-13,2013-12-23


In [9]:
import pandas as pd
from datetime import datetime, timedelta

def get_active_dates(df, service_id, only_first=True):
    """
    Returns a list of dates when a given service_id was active.
    
    Parameters:
    df (pandas.DataFrame): The calendar dataframe
    service_id (str): The service ID to look up
    
    Returns:
    list: List of datetime.date objects when the service was active
    """
    # Filter for the specific service_id
    service_row = df[df['service_id'] == service_id]
    
    if service_row.empty:
        print(f"Service ID '{service_id}' not found")
        return []
    
    # Get the first (and should be only) row
    service = service_row.iloc[0]
    
    # Parse start and end dates
    #start_date = datetime.strptime(str(service['start_date']), '%Y%m%d').date()
    #end_date = datetime.strptime(str(service['end_date']), '%Y%m%d').date()

    start_date = pd.to_datetime(service['start_date'], format='%Y%m%d')
    end_date = pd.to_datetime(service['end_date'], format='%Y%m%d')
    
    # Days of week mapping (Monday=0, Sunday=6)
    days_active = []
    day_columns = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    
    #for i, day_col in enumerate(day_columns):
    #    if service[day_col] == 1:
    #        days_active.append(i)  # Monday=0, Tuesday=1, etc.
    
    if only_first:
        days_active = [
            next((i for i, day_col in enumerate(day_columns) if service[day_col] == 1), None)
        ]
        end_date = start_date + timedelta(days=6)
    else:
        days_active = [
            i for i, day_col in enumerate(day_columns) if service[day_col] == 1
    ]
    
    if not days_active:
        print(f"Service ID '{service_id}' has no active days")
        return []
    
    # Generate all dates in the range that fall on active days
    active_dates = []
    current_date = start_date
    
    while current_date <= end_date:
        # Check if current date's weekday is in our active days
        if current_date.weekday() in days_active:
            active_dates.append(current_date)
        current_date += timedelta(days=1)
    
    return active_dates

# Example usage:
# active_dates = get_active_dates(calendar_df, 'A65119ASZGPP-021')
# print(f"Active dates: {active_dates}")
# print(f"Total active days: {len(active_dates)}")

# If you want the dates as strings instead:
def get_active_dates_str(df, service_id, date_format='%Y-%m-%d', only_first=True):
    """
    Same as get_active_dates but returns dates as formatted strings.
    """
    dates = get_active_dates(df, service_id, only_first=only_first)
    return [date.strftime(date_format) for date in dates]

# Example usage:
# active_dates_str = get_active_dates_str(calendar_df, 'A65119ASZGPP-021')
# print(active_dates_str)

In [10]:
get_active_dates_str(calendar_df, 'A69360K1HCGPP-011', only_first=False)

['2013-10-28',
 '2013-10-29',
 '2013-10-30',
 '2013-10-31',
 '2013-11-04',
 '2013-11-05',
 '2013-11-06',
 '2013-11-07',
 '2013-11-11',
 '2013-11-12',
 '2013-11-13',
 '2013-11-14',
 '2013-11-18',
 '2013-11-19',
 '2013-11-20',
 '2013-11-21',
 '2013-11-25',
 '2013-11-26',
 '2013-11-27',
 '2013-11-28',
 '2013-12-02',
 '2013-12-03',
 '2013-12-04',
 '2013-12-05',
 '2013-12-09',
 '2013-12-10',
 '2013-12-11',
 '2013-12-12',
 '2013-12-16',
 '2013-12-17',
 '2013-12-18',
 '2013-12-19',
 '2013-12-23']

In [11]:
trips_df

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,trips_bkk_ref
0,6100,A65119ASZGPP-021,A651191,Örs vezér tere M+H,1,A65119ASZGPP-021_10A,1226,2,61001
1,6100,A65119ASZGPP-021,A6511910,Gödöllő,0,A65119ASZGPP-021_10B,1225,2,61000
2,6100,A65119ASZGPP-021,A65119100,Gödöllő,0,A65119ASZGPP-021_7B,1225,2,61000
3,6100,A65119ASZGPP-021,A65119101,Örs vezér tere M+H,1,A65119ASZGPP-021_7B,1226,2,61001
4,6130,A65119ASZGPP-021,A65119102,Cinkota H,0,A65119ASZGPP-021_7B,5145,2,61300
...,...,...,...,...,...,...,...,...,...
219134,6400,B01931RAHCKP-0011,B0193195,Batthyány tér M+H,1,B01931RAHCKP-0011_15,C754,2,64001
219135,6470,B01931RAHCKP-0011,B0193196,"Békásmegyer, fordítóvágányok",0,B01931RAHCKP-0011_15,C721,2,NaN
219136,6470,B01931RAHCKP-0011,B0193197,Batthyány tér M+H,1,B01931RAHCKP-0011_15,C723,2,NaN
219137,6470,B01931RAHCKP-0011,B0193198,"Békásmegyer, fordítóvágányok",0,B01931RAHCKP-0011_15,C721,2,NaN


## Update routes_df

In [12]:
date_dt = pd.to_datetime(date, format="%Y%m%d")
# Dátum tartomány szűrése
calendar_df['start_date'] = pd.to_datetime(calendar_df['start_date'], format="%Y%m%d")
calendar_df['end_date'] = pd.to_datetime(calendar_df['end_date'], format="%Y%m%d")

print(date_dt)

# Nap neve (pl. 'friday')
day = date_dt.strftime('%A').lower()

# Egyszerű szűrés egy lépésben
valid_service_ids = calendar_df[
    (calendar_df['start_date'] <= date_dt) &
    (calendar_df['end_date'] >= date_dt) &
    (calendar_df[day] == 1)
]["service_id"].tolist()
valid_service_ids

2013-10-18 00:00:00


['A87571AHPGPP-021',
 'A88120APPCsZ-011',
 'A88303AHPHA-0011',
 'A88327AHPSzGy011',
 'A88584AHPSzGy011',
 'A88866APPGPP-021',
 'A88957APPCsZ-011',
 'A88991APPKP-0011',
 'A89527APPKP-0011',
 'A91905AHPKZ-0011',
 'A92340AHPCsZ-031',
 'A92429APPPG-0011',
 'A92884APPPG-0011',
 'A93049APPKP-0021',
 'A93473APPKZ-0011',
 'A93584APPHA-0011',
 'A93967APPPG-0011',
 'A94562APPPG-0011',
 'A94762AHPHA-0021',
 'A94769APPCsZ-011',
 'A94963APPHA-0011',
 'A94975APPCsZ-051',
 'A94975APPCsZ-061',
 'A95090APPGER-011',
 'A95129AHPKZ-0011',
 'A95149APPMA-0011',
 'A95551APPCsZ-051',
 'A95576AHPKZ-0011',
 'A95586APPKZ-0011',
 'A95633AHPKZ-0011',
 'A95640APPMA-0021',
 'A95720APPPG-0011',
 'A95730APPHA-0061',
 'A95797AHPHA-0021',
 'A95818APPSRM-021',
 'A95868APPPG-0061',
 'A95886APPCsZ-011',
 'A95886APPCsZ-021',
 'A96075APPHA-0011',
 'A96236APPHA-0011',
 'A96245AHPBA0011',
 'A96307APPBA0011',
 'A96311AHPHA-0031',
 'A96320AHPKM-0011',
 'A96395APPPG-0041',
 'A96405APPPG-0031',
 'A96435APPSRM-021',
 'A96690TJHPSzM

In [13]:
calendar_dates_df[(calendar_dates_df["date"] == int(date)) & (calendar_dates_df["exception_type"] == 2)]

,service_id,date,exception_type
1888,A98178AHPCsZ-041,20131018,2
3632,B01134APPCsZ-021,20131018,2
3682,B01188AHPKM-0011,20131018,2
4127,B01326RA1PPHZS-011,20131018,2
4131,B01331RA1PPPG-0011,20131018,2


In [14]:
remove = calendar_dates_df[(calendar_dates_df["date"] == int(date)) & (calendar_dates_df["exception_type"] == 2)]["service_id"]
remove_set = set(remove)
filtered_service_ids = [sid for sid in valid_service_ids if sid not in remove_set]

In [15]:
len(valid_service_ids), len(remove_set), len(filtered_service_ids)

(246, 5, 241)

In [16]:
calendar_df

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,A65119ASZGPP-021,0,0,0,0,0,1,0,2013-10-12,2013-10-19
1,A65120AVVGPP-021,0,0,0,0,0,0,1,2013-10-13,2013-12-22
2,A69360K1HCGPP-011,1,1,1,1,0,0,0,2013-10-28,2013-12-23
3,A69361K1PPGPP-011,0,0,0,0,1,0,0,2013-10-31,2013-10-31
4,A77793ASZSRM-021,0,0,0,0,0,1,0,2013-10-12,2013-12-14
...,...,...,...,...,...,...,...,...,...,...
1126,B01876ASZBYR-011,0,0,0,0,0,1,0,2013-10-12,2013-12-14
1127,B01879AVVBYR-011,0,0,0,0,0,0,1,2013-10-13,2013-12-22
1128,B01892ASZKP-0011,0,0,0,0,0,1,0,2013-10-12,2013-10-12
1129,B01903AVCSRM-021,1,1,1,1,0,0,1,2013-10-13,2013-12-23


In [17]:
get_active_dates_str(calendar_df, 'A69360K1HCGPP-011')

['2013-10-28']

In [18]:
trips_df["service_id"].unique()

array(['A65119ASZGPP-021', 'A65120AVVGPP-021', 'A69360K1HCGPP-011', ...,
       'B01868AVVGER-011', 'B01903AVCSRM-021', 'B01931RAHCKP-0011'],
      shape=(1127,), dtype=object)

In [19]:
trip_first_date = {}
for service in trips_df["service_id"].unique():
    if service not in trip_first_date.keys():
        first_date = get_active_dates_str(calendar_df, service, only_first=True)
        trip_first_date[service] = first_date[0]

trip_dates = {}
for service in trips_df["service_id"].unique():
    if service not in trip_dates.keys():
        dates = get_active_dates_str(calendar_df, service, only_first=False)
        trip_dates[service] = dates

In [20]:
trips_df_extended = trips_df.copy()
trips_df_extended["first_date"] = trips_df_extended["service_id"].map(trip_first_date)
trips_df_extended

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,trips_bkk_ref,first_date
0,6100,A65119ASZGPP-021,A651191,Örs vezér tere M+H,1,A65119ASZGPP-021_10A,1226,2,61001,2013-10-12
1,6100,A65119ASZGPP-021,A6511910,Gödöllő,0,A65119ASZGPP-021_10B,1225,2,61000,2013-10-12
2,6100,A65119ASZGPP-021,A65119100,Gödöllő,0,A65119ASZGPP-021_7B,1225,2,61000,2013-10-12
3,6100,A65119ASZGPP-021,A65119101,Örs vezér tere M+H,1,A65119ASZGPP-021_7B,1226,2,61001,2013-10-12
4,6130,A65119ASZGPP-021,A65119102,Cinkota H,0,A65119ASZGPP-021_7B,5145,2,61300,2013-10-12
...,...,...,...,...,...,...,...,...,...,...
219134,6400,B01931RAHCKP-0011,B0193195,Batthyány tér M+H,1,B01931RAHCKP-0011_15,C754,2,64001,2013-10-14
219135,6470,B01931RAHCKP-0011,B0193196,"Békásmegyer, fordítóvágányok",0,B01931RAHCKP-0011_15,C721,2,NaN,2013-10-14
219136,6470,B01931RAHCKP-0011,B0193197,Batthyány tér M+H,1,B01931RAHCKP-0011_15,C723,2,NaN,2013-10-14
219137,6470,B01931RAHCKP-0011,B0193198,"Békásmegyer, fordítóvágányok",0,B01931RAHCKP-0011_15,C721,2,NaN,2013-10-14


In [21]:
trips_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219139 entries, 0 to 219138
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   route_id               219139 non-null  object
 1   service_id             219139 non-null  object
 2   trip_id                219139 non-null  object
 3   trip_headsign          219139 non-null  object
 4   direction_id           219139 non-null  int64 
 5   block_id               219139 non-null  object
 6   shape_id               219139 non-null  object
 7   wheelchair_accessible  219139 non-null  int64 
 8   trips_bkk_ref          188074 non-null  object
dtypes: int64(2), object(7)
memory usage: 15.0+ MB


In [22]:
trips_df_extended.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219139 entries, 0 to 219138
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   route_id               219139 non-null  object
 1   service_id             219139 non-null  object
 2   trip_id                219139 non-null  object
 3   trip_headsign          219139 non-null  object
 4   direction_id           219139 non-null  int64 
 5   block_id               219139 non-null  object
 6   shape_id               219139 non-null  object
 7   wheelchair_accessible  219139 non-null  int64 
 8   trips_bkk_ref          188074 non-null  object
 9   first_date             219139 non-null  object
dtypes: int64(2), object(8)
memory usage: 16.7+ MB


In [23]:
#trips2routes_df = trips_df[trips_df["service_id"].isin(valid_service_ids)]
trips2routes_df = trips_df_extended.copy()
trips2routes_df = trips2routes_df[["service_id", "route_id", "shape_id", "trip_headsign", "direction_id", "trips_bkk_ref", "first_date"]]
#trips2routes_df = trips2routes_df[trips2routes_df["trips_bkk_ref"].notna()]
trips2routes_df = trips2routes_df.groupby(["route_id", "shape_id", "trip_headsign", "direction_id", "first_date"]).count().reset_index()
trips2routes_df = trips2routes_df.sort_values(by=['route_id', 'direction_id', 'trips_bkk_ref', 'service_id'], ascending=[True, True, False, False])
trips2routes_df = trips2routes_df.drop_duplicates(subset=['route_id', 'direction_id'], ignore_index=True)
trips2routes_df = trips2routes_df.rename(columns={"shape_id" : "main_shape_id", "first_date" : "valid_from"})
trips2routes_df

,route_id,main_shape_id,trip_headsign,direction_id,valid_from,service_id,trips_bkk_ref
0,0050,R952,"Rákospalota, Kossuth utca",0,2013-10-11,112,112
1,0050,U961,Pasaréti tér,1,2013-10-11,114,114
2,0070,S704,"Újpalota, Nyírpalota út",0,2013-10-11,145,145
3,0070,S705,Albertfalva vasútállomás,1,2013-10-11,145,145
4,0071,S710,Bosnyák tér,0,2013-10-21,254,254
...,...,...,...,...,...,...,...
616,VP101,X064,"Óbuda, Bogdáni út",1,2013-10-11,243,243
617,VP42,X162,"Kispest, Tulipán utca",0,2013-10-11,114,114
618,VP42,X163,Határ út M,1,2013-10-11,115,115
619,VP61,X017,Hűvösvölgy,0,2013-10-11,285,285


In [24]:
trips2routes_df.sort_values("trips_bkk_ref", ascending=True)

,route_id,main_shape_id,trip_headsign,direction_id,valid_from,service_id,trips_bkk_ref
602,9980,M118,Cinkotai autóbuszgarázs,1,2013-10-11,5,0
80,0540,M128,Boráros tér H,1,2013-10-12,33,0
82,0550,M130,Boráros tér H,1,2013-10-14,41,0
74,0450,A067,Örs vezér tere M+H,1,2013-10-11,88,0
75,0460,T413,"Városközpont, Posta",0,2013-10-14,43,0
...,...,...,...,...,...,...,...
14,0110,0715,Batthyány tér M+H,1,2013-10-14,376,376
498,5100,1109,Vörösmarty tér,1,2013-10-11,393,393
497,5100,1110,Mexikói út M,0,2013-10-11,393,393
607,MP525,V038,Örs vezér tere M+H,0,2013-10-12,404,404


In [25]:
trips2routes_df[trips2routes_df["route_id"] == "0450"]

,route_id,main_shape_id,trip_headsign,direction_id,valid_from,service_id,trips_bkk_ref
73,0450,A066,"Cinkota, Lassú utca",0,2013-10-11,86,86
74,0450,A067,Örs vezér tere M+H,1,2013-10-11,88,0


In [26]:
latest_routes_df = pd.merge(
    #routes_txt[["route_id", "route_long_name" ,"route_desc"]],
    routes_txt,
    trips2routes_df[["route_id", "main_shape_id", "trip_headsign", "direction_id", "valid_from"]],
    on="route_id",
    how="inner",)

In [27]:
latest_routes_df

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_text_color,main_shape_id,trip_headsign,direction_id,valid_from
0,MP52,BKK,M2,NaN,Széll Kálmán tér M / Örs vezér tere M+H,3,1188FF,000000,V040,Örs vezér tere M+H,0,2013-10-12
1,MP52,BKK,M2,NaN,Széll Kálmán tér M / Örs vezér tere M+H,3,1188FF,000000,V041,Széll Kálmán tér M,1,2013-10-12
2,MP525,BKK,M2E,NaN,Széll Kálmán tér M / Örs vezér tere M+H,3,1188FF,000000,V038,Örs vezér tere M+H,0,2013-10-12
3,MP525,BKK,M2E,NaN,Széll Kálmán tér M / Örs vezér tere M+H,3,1188FF,000000,V039,Széll Kálmán tér M,1,2013-10-12
4,VP01,BKK,1V,NaN,Bécsi út (Vörösvári út) / Lehel utca/Róbert Ká...,3,1188FF,000000,X032,Lehel utca/Róbert Károly körút,0,2013-10-11
...,...,...,...,...,...,...,...,...,...,...,...,...
616,9961,BKK,996A,NaN,Cinkotai autóbuszgarázs / Újpest-Központ M,3,333333,FFFFFF,C182,Újpest-Központ M,1,2013-10-11
617,9980,BKK,998,NaN,"Rákoskeresztúr, városközpont / Rákoskeresztúr,...",3,333333,FFFFFF,M116,"Rákoskeresztúr, városközpont",0,2013-10-11
618,9980,BKK,998,NaN,"Rákoskeresztúr, városközpont / Rákoskeresztúr,...",3,333333,FFFFFF,M118,Cinkotai autóbuszgarázs,1,2013-10-11
619,9990,BKK,999,NaN,Határ út M / Dél-pesti autóbuszgarázs,3,333333,FFFFFF,Q605,Dél-pesti autóbuszgarázs,0,2013-10-11


In [28]:
latest_routes_df[latest_routes_df["route_id"] == "3060"]

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_text_color,main_shape_id,trip_headsign,direction_id,valid_from
433,3060,BKK,6,NaN,Széll Kálmán tér M / Móricz Zsigmond körtér,0,FFEE00,000000,B693,Móricz Zsigmond körtér,0,2013-10-11
434,3060,BKK,6,NaN,Széll Kálmán tér M / Móricz Zsigmond körtér,0,FFEE00,000000,B692,Széll Kálmán tér M,1,2013-10-11


In [29]:
routes_df

,route_id,agency_id,route_short_name,route_type,route_color,route_text_color
0,MP52,BKK,M2,3,1188FF,000000
1,MP52,BKK,M2,3,1188FF,000000
2,MP525,BKK,M2E,3,1188FF,000000
3,MP525,BKK,M2E,3,1188FF,000000
4,VP01,BKK,1V,3,1188FF,000000
...,...,...,...,...,...,...
616,9961,BKK,996A,3,333333,FFFFFF
617,9980,BKK,998,3,333333,FFFFFF
618,9980,BKK,998,3,333333,FFFFFF
619,9990,BKK,999,3,333333,FFFFFF


In [30]:
def update_routes_df(routes_df, latest_routes_df):
    # Use relevant columns, without route_desc
    cols_to_use = [col for col in routes_df.columns]

    # Select new rows - rows whats route_id is not in routes_df 
    new_routes = latest_routes_df[~latest_routes_df["route_id"].isin(routes_df["route_id"])][cols_to_use]
    
    # Concatenate new routes
    updated_routes_df = pd.concat([routes_df, new_routes], ignore_index=True)

    # Check for duplicates
    #duplicates = updated_routes_df[updated_routes_df.duplicated(subset="route_id", keep=False)]
    duplicates = updated_routes_df[updated_routes_df.groupby("route_id")["route_id"].transform("count") > 2]

    if not duplicates.empty:
        print(f"Warning: There are {duplicates['route_id'].nunique()} duplicated route_id(s) in routes_df!")
        print("Duplicated route_id(s):")
        print(duplicates['route_id'].unique())
    else:
        print("No duplicate route_id found in routes_df.")

    return updated_routes_df

def save_routes(routes_df, data_folder):
    _, routes_df_path, _, _, _, _ = define_paths(data_folder=data_folder)
    routes_df.to_csv(routes_df_path, index=False)
    print(f"routes_df saved to {routes_df_path}")

In [31]:
updated_routes_df = update_routes_df(routes_df, latest_routes_df)
save_routes(updated_routes_df, data_folder)

No duplicate route_id found in routes_df.
routes_df saved to ../data/processed/routes.csv


In [32]:
updated_routes_df

,route_id,agency_id,route_short_name,route_type,route_color,route_text_color
0,MP52,BKK,M2,3,1188FF,000000
1,MP52,BKK,M2,3,1188FF,000000
2,MP525,BKK,M2E,3,1188FF,000000
3,MP525,BKK,M2E,3,1188FF,000000
4,VP01,BKK,1V,3,1188FF,000000
...,...,...,...,...,...,...
616,9961,BKK,996A,3,333333,FFFFFF
617,9980,BKK,998,3,333333,FFFFFF
618,9980,BKK,998,3,333333,FFFFFF
619,9990,BKK,999,3,333333,FFFFFF


In [33]:
updated_routes_df[updated_routes_df["route_id"] == "3190"]

,route_id,agency_id,route_short_name,route_type,route_color,route_text_color
443,3190,BKK,19,0,FFEE00,000000
444,3190,BKK,19,0,FFEE00,000000


## Update route_versions_df

In [34]:
route_versions_df

,version_id,route_id,direction_id,route_long_name,route_desc,valid_from,valid_to,main_shape_id,trip_headsign,parent_version_id,note
0,100000,MP52,0,NaN,Széll Kálmán tér M / Örs vezér tere M+H,2013-10-12,NaT,V040,Örs vezér tere M+H,NaN,NaN
1,100001,MP52,1,NaN,Széll Kálmán tér M / Örs vezér tere M+H,2013-10-12,NaT,V041,Széll Kálmán tér M,NaN,NaN
2,100002,MP525,0,NaN,Széll Kálmán tér M / Örs vezér tere M+H,2013-10-12,NaT,V038,Örs vezér tere M+H,NaN,NaN
3,100003,MP525,1,NaN,Széll Kálmán tér M / Örs vezér tere M+H,2013-10-12,NaT,V039,Széll Kálmán tér M,NaN,NaN
4,100004,VP01,0,NaN,Bécsi út (Vörösvári út) / Lehel utca/Róbert Ká...,2013-10-11,NaT,X032,Lehel utca/Róbert Károly körút,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
616,100616,9961,1,NaN,Cinkotai autóbuszgarázs / Újpest-Központ M,2013-10-11,NaT,C182,Újpest-Központ M,NaN,NaN
617,100617,9980,0,NaN,"Rákoskeresztúr, városközpont / Rákoskeresztúr,...",2013-10-11,NaT,M116,"Rákoskeresztúr, városközpont",NaN,NaN
618,100618,9980,1,NaN,"Rákoskeresztúr, városközpont / Rákoskeresztúr,...",2013-10-11,NaT,M118,Cinkotai autóbuszgarázs,NaN,NaN
619,100619,9990,0,NaN,Határ út M / Dél-pesti autóbuszgarázs,2013-10-11,NaT,Q605,Dél-pesti autóbuszgarázs,NaN,NaN


In [35]:
def version_exists(current_versions, row):
    return (
        ((current_versions["route_id"] == row["route_id"]) &
         (current_versions["direction_id"] == row["direction_id"]) &
         (current_versions["main_shape_id"] == row["main_shape_id"]) &
         (current_versions["trip_headsign"] == row["trip_headsign"]))
        .any()
    )

def update_route_versions(route_versions_df, latest_routes_df, date):
    route_versions_copy_df = route_versions_df.copy()
    # version_id starting point
    START_VERSION_ID = 100_000

    # If the file is empty
    if route_versions_df.empty:
        next_version_id = START_VERSION_ID
    else:
        next_version_id = route_versions_df["version_id"].max() + 1

    # Prepare new versions
    #trips_grouped = trips_txt.groupby(['route_id', 'shape_id', 'trip_headsign', 'direction_id']).count()
    #trips_grouped = trips_grouped.sort_values(by=['route_id', 'service_id'], ascending=[True, False])
    #trips_grouped = trips_grouped.groupby('route_id').head(2).sort_values(by=['route_id', 'direction_id']).reset_index()
    #trips_grouped = trips_grouped[["route_id", "shape_id", "trip_headsign", "direction_id"]]

    # Create a new versions dataframe
    #new_versions_df = pd.merge(trips_grouped, routes_txt[["route_id", "route_long_name", "route_desc"]], on="route_id")
    new_versions_df = latest_routes_df.copy()[["route_id", "main_shape_id", "trip_headsign", "direction_id", "route_desc", "valid_from"]]
    new_versions_df["valid_from"] = pd.to_datetime(new_versions_df['valid_from'])
    new_versions_df["valid_to"] = pd.NaT
    new_versions_df["parent_version_id"] = np.nan
    new_versions_df["note"] = np.nan

    # Define the current versions
    current_versions = route_versions_df[route_versions_df["valid_to"].isna()]

    # Let only the new versions
    ##new_versions_filtered = new_versions_df[~new_versions_df.apply(version_exists, axis=1)].copy()
    new_versions_filtered = new_versions_df[~new_versions_df.apply(lambda row: version_exists(row, current_versions), axis=1)].copy()

    # Update the previous versions valid_to date
    for _, row in new_versions_filtered.iterrows():
        mask = (
            (route_versions_df["route_id"] == row["route_id"]) &
            (route_versions_df["valid_to"].isna())
        )
        route_versions_copy_df.loc[mask, "valid_to"] = row["valid_from"] - pd.Timedelta(days=1)

    new_versions_filtered["version_id"] = range(next_version_id, next_version_id + len(new_versions_filtered))

    # Concat
    extended_route_versions_df = pd.concat([route_versions_copy_df, new_versions_filtered], ignore_index=True)

    return extended_route_versions_df

def save_route_versions(route_versions_df, data_folder):
    _, _, route_versions_df_path, _, _, _ = define_paths(data_folder=data_folder)
    route_versions_df.to_csv(route_versions_df_path, index=False)
    print(f"routes_df saved to {route_versions_df_path}")

In [36]:
extended_route_versions_df = update_route_versions(route_versions_df, latest_routes_df, date)
save_route_versions(extended_route_versions_df, data_folder)
extended_route_versions_df

routes_df saved to ../data/processed/route_versions.csv


,version_id,route_id,direction_id,route_long_name,route_desc,valid_from,valid_to,main_shape_id,trip_headsign,parent_version_id,note
0,100000,MP52,0,NaN,Széll Kálmán tér M / Örs vezér tere M+H,2013-10-12,NaT,V040,Örs vezér tere M+H,NaN,NaN
1,100001,MP52,1,NaN,Széll Kálmán tér M / Örs vezér tere M+H,2013-10-12,NaT,V041,Széll Kálmán tér M,NaN,NaN
2,100002,MP525,0,NaN,Széll Kálmán tér M / Örs vezér tere M+H,2013-10-12,NaT,V038,Örs vezér tere M+H,NaN,NaN
3,100003,MP525,1,NaN,Széll Kálmán tér M / Örs vezér tere M+H,2013-10-12,NaT,V039,Széll Kálmán tér M,NaN,NaN
4,100004,VP01,0,NaN,Bécsi út (Vörösvári út) / Lehel utca/Róbert Ká...,2013-10-11,NaT,X032,Lehel utca/Róbert Károly körút,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
616,100616,9961,1,NaN,Cinkotai autóbuszgarázs / Újpest-Központ M,2013-10-11,NaT,C182,Újpest-Központ M,NaN,NaN
617,100617,9980,0,NaN,"Rákoskeresztúr, városközpont / Rákoskeresztúr,...",2013-10-11,NaT,M116,"Rákoskeresztúr, városközpont",NaN,NaN
618,100618,9980,1,NaN,"Rákoskeresztúr, városközpont / Rákoskeresztúr,...",2013-10-11,NaT,M118,Cinkotai autóbuszgarázs,NaN,NaN
619,100619,9990,0,NaN,Határ út M / Dél-pesti autóbuszgarázs,2013-10-11,NaT,Q605,Dél-pesti autóbuszgarázs,NaN,NaN


In [37]:
#extended_route_versions_df.groupby("route_id").count()["version_id"].to_dict()

In [38]:
extended_route_versions_df[extended_route_versions_df["route_id"] == "3060"]

,version_id,route_id,direction_id,route_long_name,route_desc,valid_from,valid_to,main_shape_id,trip_headsign,parent_version_id,note
433,100433,3060,0,NaN,Széll Kálmán tér M / Móricz Zsigmond körtér,2013-10-11,NaT,B693,Móricz Zsigmond körtér,NaN,NaN
434,100434,3060,1,NaN,Széll Kálmán tér M / Móricz Zsigmond körtér,2013-10-11,NaT,B692,Széll Kálmán tér M,NaN,NaN


## Update shape_variants_df

### First step update

In [39]:
calendar_dates_df['date'] = pd.to_datetime(calendar_dates_df['date'], format="%Y%m%d")

#extra_service_ids = calendar_dates_df[
#    (calendar_dates_df['date'] == date_dt)
#][["service_id", "exception_type"]]
extra_service_ids = calendar_dates_df[["date", "service_id", "exception_type"]].copy()
extra_service_ids["exception_type"] = extra_service_ids["exception_type"].astype(int)

In [40]:
extra_service_ids

,date,service_id,exception_type
0,2013-10-23,A65120AVVGPP-021,1
1,2013-11-01,A65120AVVGPP-021,1
2,2013-10-31,A69360K1HCGPP-011,2
3,2013-12-06,A69360K1HCGPP-011,1
4,2013-12-20,A69360K1HCGPP-011,1
...,...,...,...
4872,2013-10-31,B01903AVCSRM-021,2
4873,2013-11-01,B01903AVCSRM-021,1
4874,2013-12-06,B01903AVCSRM-021,1
4875,2013-12-20,B01903AVCSRM-021,1


In [41]:
valid_routes = extended_route_versions_df[extended_route_versions_df["valid_to"].isna()][["version_id", "route_id", "direction_id", "main_shape_id"]]
valid_routes

,version_id,route_id,direction_id,main_shape_id
0,100000,MP52,0,V040
1,100001,MP52,1,V041
2,100002,MP525,0,V038
3,100003,MP525,1,V039
4,100004,VP01,0,X032
...,...,...,...,...
616,100616,9961,1,C182
617,100617,9980,0,M116
618,100618,9980,1,M118
619,100619,9990,0,Q605


In [42]:
non_empty_keys = [key for key, value in trip_dates.items() if value]
len(trip_dates.values()), len(non_empty_keys)

(1127, 1088)

In [43]:
actual_valid_service_ids = non_empty_keys

In [44]:
trips2routes_df2 = trips_df[trips_df['service_id'].isin(non_empty_keys)]
trips2routes_df2 = trips2routes_df2[["service_id", "route_id", "shape_id", "trip_headsign", "direction_id"]]
#trips2routes_df2["date"] = trips2routes_df2["service_id"].map(trip_dates)
trips2routes_df2 = trips2routes_df2.groupby(["route_id", "shape_id", "trip_headsign", "direction_id"]).agg('first').reset_index()
#trips2routes_df2 = trips2routes_df2[["date", "route_id", "shape_id", "trip_headsign", "direction_id", "exception_type"]]
trips2routes_df2

,route_id,shape_id,trip_headsign,direction_id,service_id
0,0050,R952,"Rákospalota, Kossuth utca",0,A99694AVVKZ-0031
1,0050,U197,"Rákospalota, Kossuth utca",0,A99694AVVKZ-0031
2,0050,U961,Pasaréti tér,1,A99694AVVKZ-0031
3,0070,S704,"Újpalota, Nyírpalota út",0,B00827AHCPG-0061
4,0070,S705,Albertfalva vasútállomás,1,B00827AHCPG-0061
...,...,...,...,...,...
1146,VP101,X064,"Óbuda, Bogdáni út",1,B01253RA1HCHZS-011
1147,VP42,X162,"Kispest, Tulipán utca",0,B01475RA1SZPG-0011
1148,VP42,X163,Határ út M,1,B01475RA1SZPG-0011
1149,VP61,X016,Széll Kálmán tér M,1,B01269RA1HCPG-0011


In [45]:
# First add the list column
df_temp = trips2routes_df2.copy()
df_temp['date_list'] = df_temp['service_id'].map(trip_dates)

# Then explode the list column into separate rows
df_exploded = df_temp.explode('date_list')
df_exploded = df_exploded.rename(columns={'date_list': 'date'})
df_exploded.drop(columns=['service_id'], inplace=True)
df_exploded['exception_type'] = np.nan
print("After exploding:")
df_exploded

After exploding:


,route_id,shape_id,trip_headsign,direction_id,date,exception_type
0,0050,R952,"Rákospalota, Kossuth utca",0,2013-10-27,NaN
0,0050,R952,"Rákospalota, Kossuth utca",0,2013-11-03,NaN
0,0050,R952,"Rákospalota, Kossuth utca",0,2013-11-10,NaN
0,0050,R952,"Rákospalota, Kossuth utca",0,2013-11-17,NaN
0,0050,R952,"Rákospalota, Kossuth utca",0,2013-11-24,NaN
...,...,...,...,...,...,...
1150,VP61,X017,Hűvösvölgy,0,2013-12-16,NaN
1150,VP61,X017,Hűvösvölgy,0,2013-12-17,NaN
1150,VP61,X017,Hűvösvölgy,0,2013-12-18,NaN
1150,VP61,X017,Hűvösvölgy,0,2013-12-19,NaN


### Second step update

In [46]:
extra_service_ids

,date,service_id,exception_type
0,2013-10-23,A65120AVVGPP-021,1
1,2013-11-01,A65120AVVGPP-021,1
2,2013-10-31,A69360K1HCGPP-011,2
3,2013-12-06,A69360K1HCGPP-011,1
4,2013-12-20,A69360K1HCGPP-011,1
...,...,...,...
4872,2013-10-31,B01903AVCSRM-021,2
4873,2013-11-01,B01903AVCSRM-021,1
4874,2013-12-06,B01903AVCSRM-021,1
4875,2013-12-20,B01903AVCSRM-021,1


In [47]:
trips2routes_df3 = pd.merge(trips_df, extra_service_ids, how="left", on="service_id")
trips2routes_df3 = trips2routes_df3.groupby(["route_id", "shape_id", "trip_headsign", "direction_id", "date"]).agg('first').reset_index()
trips2routes_df3 = trips2routes_df3[["date", "route_id", "shape_id", "trip_headsign", "direction_id", "exception_type"]]

# This will actually convert 2.0 to "2" and keep NaN as NaN
trips2routes_df3["exception_type"] = trips2routes_df3["exception_type"].apply(
    lambda x: str(int(x)) if pd.notna(x) else x
)

trips2routes_df3

,date,route_id,shape_id,trip_headsign,direction_id,exception_type
0,2013-10-23,0050,R952,"Rákospalota, Kossuth utca",0,1
1,2013-10-27,0050,R952,"Rákospalota, Kossuth utca",0,2
2,2013-11-01,0050,R952,"Rákospalota, Kossuth utca",0,1
3,2013-11-08,0050,R952,"Rákospalota, Kossuth utca",0,2
4,2013-11-15,0050,R952,"Rákospalota, Kossuth utca",0,2
...,...,...,...,...,...,...
17601,2013-11-01,VP61,X017,Hűvösvölgy,0,2
17602,2013-12-06,VP61,X017,Hűvösvölgy,0,1
17603,2013-12-07,VP61,X017,Hűvösvölgy,0,2
17604,2013-12-20,VP61,X017,Hűvösvölgy,0,1


### Third step: merge updates

In [48]:
import pandas as pd

df1 = trips2routes_df3.copy()
df2 = df_exploded.copy()

# Egységes date formátum mindkettőben
df1['date'] = pd.to_datetime(df1['date']).dt.strftime('%Y-%m-%d')
df2['date'] = pd.to_datetime(df2['date']).dt.strftime('%Y-%m-%d')

# Összefűzzük a kettőt
combined = pd.concat([df1, df2], ignore_index=True)

# Az oszlopok, amelyek alapján duplikációt nézünk (exception_type nélkül)
cols_except_exception = [col for col in combined.columns if col != 'exception_type']

# Megjelöljük, hogy az exception_type NaN-e
combined['__is_nan__'] = combined['exception_type'].isna()

# Rendezés: nem-NaN előre
combined_sorted = combined.sort_values('__is_nan__')

# Duplikátumok kiszűrése
before = len(combined_sorted)
final_df = combined_sorted.drop_duplicates(subset=cols_except_exception, keep='first')
after = len(final_df)
removed = before - after

# Segédoszlop eltávolítása
final_df = final_df.drop(columns='__is_nan__')

# Rendezés a megadott oszlopok szerint
sort_columns = ['date', 'route_id', 'direction_id', 'shape_id', 'trip_headsign', 'exception_type']
final_df = final_df.sort_values(by=sort_columns)

# Index újraszámozása
final_df = final_df.reset_index(drop=True)

# Jelentés
print(f"{removed} duplikált sort eltávolítottunk, ahol csak az exception_type tért el (NaN vs nem-NaN).")


7629 duplikált sort eltávolítottunk, ahol csak az exception_type tért el (NaN vs nem-NaN).


In [49]:
final_df

,date,route_id,shape_id,trip_headsign,direction_id,exception_type
0,2013-10-11,0071,S710,Bosnyák tér,0,NaN
1,2013-10-11,0071,S713,Bosnyák tér,0,NaN
2,2013-10-11,0071,U871,Bosnyák tér,0,NaN
3,2013-10-11,0071,S711,Bornemissza tér,1,NaN
4,2013-10-11,0250,U542,Újpest-Központ M,0,NaN
...,...,...,...,...,...,...
38749,2013-12-23,9990,Q606,Határ út M,1,NaN
38750,2013-12-23,VP101,X034,Lehel utca/Róbert Károly körút,0,NaN
38751,2013-12-23,VP101,X064,"Óbuda, Bogdáni út",1,NaN
38752,2013-12-23,VP61,X017,Hűvösvölgy,0,NaN


In [50]:
#final_df[final_df["route_id"] == "0050"]

In [51]:
final_df["exception_type"].unique()

array([nan, '2', '1'], dtype=object)

### Fourth step: final merge

In [52]:
merged_df = pd.merge(valid_routes, final_df, on=["route_id", "direction_id"])
merged_df["main_shape_id"] = (merged_df["main_shape_id"] == merged_df["shape_id"]).astype(int)
merged_df = merged_df.rename(columns={"main_shape_id" : "is_main"})
merged_df

,version_id,route_id,direction_id,is_main,date,shape_id,trip_headsign,exception_type
0,100000,MP52,0,1,2013-10-13,V040,Örs vezér tere M+H,NaN
1,100000,MP52,0,0,2013-10-17,X067,Örs vezér tere M+H,NaN
2,100000,MP52,0,1,2013-10-20,V040,Örs vezér tere M+H,NaN
3,100000,MP52,0,0,2013-10-21,X067,Örs vezér tere M+H,NaN
4,100000,MP52,0,0,2013-10-22,X067,Örs vezér tere M+H,NaN
...,...,...,...,...,...,...,...,...
38749,100620,9990,1,1,2013-12-19,Q606,Határ út M,NaN
38750,100620,9990,1,1,2013-12-20,Q606,Határ út M,1
38751,100620,9990,1,1,2013-12-21,Q606,Határ út M,1
38752,100620,9990,1,1,2013-12-22,Q606,Határ út M,NaN


In [53]:
#merged_df[merged_df["exception_type"].isna()]

In [54]:
shape_variants_df

,shape_variant_id,version_id,shape_id,trip_headsign,is_main,note


In [55]:
shape_variant_activations_df

,date,shape_variant_id,exception_type


In [56]:
import pandas as pd

# Get unique shape variants from merged_df
new_variants = merged_df[['version_id', 'shape_id', 'trip_headsign', 'is_main']].drop_duplicates().reset_index(drop=True)

# Check which variants are already in shape_variants_df
if not shape_variants_df.empty:
    existing_variants = shape_variants_df[['version_id', 'shape_id', 'trip_headsign', 'is_main']]
    # Find variants that don't already exist
    merged_check = new_variants.merge(
        existing_variants, 
        on=['version_id', 'shape_id', 'trip_headsign', 'is_main'], 
        how='left', 
        indicator=True
    )
    truly_new_variants = merged_check[merged_check['_merge'] == 'left_only'].drop('_merge', axis=1).reset_index(drop=True)
else:
    truly_new_variants = new_variants

# Add new variants to shape_variants_df
if not truly_new_variants.empty:
    # Determine starting shape_variant_id
    if shape_variants_df.empty:
        start_id = 100000
    else:
        start_id = shape_variants_df['shape_variant_id'].max() + 1
    
    # Create new variant records
    new_variant_records = truly_new_variants.copy()
    new_variant_records['shape_variant_id'] = range(start_id, start_id + len(truly_new_variants))
    new_variant_records['note'] = None
    new_variant_records = new_variant_records[['shape_variant_id', 'version_id', 'shape_id', 'trip_headsign', 'is_main', 'note']]
    
    # Append to existing shape_variants_df
    shape_variants_df = pd.concat([shape_variants_df, new_variant_records], ignore_index=True)

# Create mapping for all variants (existing + new)
variant_mapping = shape_variants_df[['shape_variant_id', 'version_id', 'shape_id', 'trip_headsign', 'is_main']].copy()

# Merge merged_df with variant mapping to get shape_variant_id for each row
merged_with_variant_id = merged_df.merge(
    variant_mapping, 
    on=['version_id', 'shape_id', 'trip_headsign', 'is_main'], 
    how='left'
)

# Create new activation records
new_activations = merged_with_variant_id[['date', 'shape_variant_id', 'exception_type']].copy()

# Check which activations are already in shape_variant_activations_df
if not shape_variant_activations_df.empty:
    # Find activations that don't already exist
    merged_activations_check = new_activations.merge(
        shape_variant_activations_df, 
        on=['date', 'shape_variant_id', 'exception_type'], 
        how='left', 
        indicator=True
    )
    truly_new_activations = merged_activations_check[merged_activations_check['_merge'] == 'left_only'].drop('_merge', axis=1).reset_index(drop=True)
else:
    truly_new_activations = new_activations

# Add new activations to shape_variant_activations_df
if not truly_new_activations.empty:
    shape_variant_activations_df = pd.concat([shape_variant_activations_df, truly_new_activations], ignore_index=True)

shape_variant_activations_df.sort_values(['date', 'shape_variant_id'], inplace=True)
shape_variant_activations_df.reset_index(drop=True, inplace=True)

# Display results
print("Updated shape_variants_df:")
#print(shape_variants_df.head(10))
print(f"Shape: {shape_variants_df.shape}")
print()

print("Updated shape_variant_activations_df:")
#print(shape_variant_activations_df.head(10))
print(f"Shape: {shape_variant_activations_df.shape}")

# Summary
print(f"\nSummary:")
print(f"Total unique shape variants: {len(shape_variants_df)}")
print(f"Total shape variant activations: {len(shape_variant_activations_df)}")
if not truly_new_variants.empty:
    print(f"New variants added: {len(truly_new_variants)}")
    print(f"Shape variant IDs added: {truly_new_variants['shape_variant_id'].min()} - {truly_new_variants['shape_variant_id'].max()}" if 'shape_variant_id' in locals() else "")
else:
    print("No new variants added")
if not truly_new_activations.empty:
    print(f"New activations added: {len(truly_new_activations)}")
else:
    print("No new activations added")

Updated shape_variants_df:
Shape: (1151, 6)

Updated shape_variant_activations_df:
Shape: (38754, 3)

Summary:
Total unique shape variants: 1151
Total shape variant activations: 38754
New variants added: 1151

New activations added: 38754


In [57]:
shape_variant_activations_df.head(20)

,date,shape_variant_id,exception_type
0,2013-10-11,100028,NaN
1,2013-10-11,100029,NaN
2,2013-10-11,100030,NaN
3,2013-10-11,100031,NaN
4,2013-10-11,100068,NaN
5,2013-10-11,100069,NaN
6,2013-10-11,100070,NaN
7,2013-10-11,100116,NaN
8,2013-10-11,100119,NaN
9,2013-10-11,100120,NaN


In [179]:
##### ITT TARTOK #####

In [176]:
shape_variants_df

,shape_variant_id,version_id,shape_id,trip_headsign,is_main,note
0,0,100000,V040,Örs vezér tere M+H,1,None
1,1,100000,X067,Örs vezér tere M+H,0,None
2,2,100001,V041,Széll Kálmán tér M,1,None
3,3,100001,X068,Széll Kálmán tér M,0,None
4,4,100002,V038,Örs vezér tere M+H,1,None
...,...,...,...,...,...,...
1146,1146,100617,M116,"Rákoskeresztúr, városközpont",1,None
1147,1147,100618,M117,"Rákoskeresztúr, városközpont",0,None
1148,1148,100618,M118,Cinkotai autóbuszgarázs,1,None
1149,1149,100619,Q605,Dél-pesti autóbuszgarázs,1,None


In [174]:
shape_variant_activations_df

,date,shape_variant_id,exception_type
0,2013-10-13,0,NaN
1,2013-10-17,1,NaN
2,2013-10-20,0,NaN
3,2013-10-21,1,NaN
4,2013-10-22,1,NaN
...,...,...,...
38749,2013-12-19,1150,NaN
38750,2013-12-20,1150,1
38751,2013-12-21,1150,1
38752,2013-12-22,1150,NaN


In [59]:
def save_shape_variants_df(shape_variants_df, data_folder):
    _, _, _, shape_variants_df_path, _, _ = define_paths(data_folder=data_folder)
    shape_variants_df.to_csv(shape_variants_df_path, index=False)
    print(f"shape_variants_df saved to {shape_variants_df_path}")

def save_shape_variant_activations_df(shape_variant_activations_df, data_folder):
    _, _, _, _, shape_variant_activations_df_path, _ = define_paths(data_folder=data_folder)
    shape_variant_activations_df.to_csv(shape_variant_activations_df_path, index=False)
    print(f"shape_variant_activations_df saved to {shape_variant_activations_df_path}")

In [60]:
save_shape_variants_df(shape_variants_df, data_folder)
save_shape_variant_activations_df(shape_variant_activations_df, data_folder)

shape_variants_df saved to ../data/processed/shape_variants.csv
shape_variant_activations_df saved to ../data/processed/shape_variant_activations.csv
